In [39]:
import numpy as np
import pandas as pd
from quasarpy.quasar import Quasar, DatasetConfig, KrigingConfig

## 1. Generate Synthetic Data

We will use the same data as the integration test:
$$ y(t) = (2x_1 + x_2) \cdot (1 + 0.5t) $$

Where:
- $t \in [0, 2]$


In [40]:
# Define the functions from the integration test
def y1_func(t, x1, x2):
    return (2 * x1 + x2) * (1 + 0.5 * t)

def y2_func(t, x1, x2):
    return (3 * x1 - x2) * np.exp(-0.3 * t)

# Time steps
t_steps = np.linspace(0, 2, 21)

def generate_data_from_func(X_df, func, noise_std=0.0):
    y_data = {}
    for i in range(len(X_df)):
        row = X_df.iloc[i]
        curve = func(t_steps, row['x1'], row['x2'])
        if noise_std > 0:
            # Add Gaussian noise
            curve += np.random.normal(0, noise_std, size=curve.shape)
        y_data[i] = curve
    return pd.DataFrame(y_data).T

# Training Data (Same as test_quasar_integration)
X_train = pd.DataFrame({
    'x1': [1.0, 2.0, 3.0, 4.0, 5.0],
    'x2': [1.0, 2.0, 1.0, 2.0, 1.0]
})
Y_train_1 = generate_data_from_func(X_train, y1_func, noise_std=0.0)
Y_train_2 = generate_data_from_func(X_train, y2_func, noise_std=0.0)

print(f"Training Samples: {len(X_train)}")
print(X_train)

Training Samples: 5
    x1   x2
0  1.0  1.0
1  2.0  2.0
2  3.0  1.0
3  4.0  2.0
4  5.0  1.0


## 2. Train the Model

We configure a dataset and train the Quasar model.

In [41]:
# Configure Datasets
ds_config_1 = DatasetConfig(
    name='Dataset1',
    data=Y_train_1,
    kriging_config=KrigingConfig(basis_function=2) # Linear basis
)

ds_config_2 = DatasetConfig(
    name='Dataset2',
    data=Y_train_2,
    kriging_config=KrigingConfig(basis_function=2) # Linear basis
)

# Initialize Quasar
# Ensure ODYSSEE_CAE_INSTALLDIR is set in your environment
try:
    q = Quasar()
    print("Quasar initialized successfully.")
except FileNotFoundError as e:
    print(e)
    print("Please set ODYSSEE_CAE_INSTALLDIR to run this demo.")

# Train
if 'q' in locals():
    q.train(X_train, [ds_config_1, ds_config_2])
    print("Training complete.")

Quasar initialized successfully.
Training complete.


## 3. Validation

We generate a new set of data to validate the model's performance.

In [42]:
# Generate Validation Data (Intermediate points with Noise)
# Increase number of samples for better visualization
X_val = pd.DataFrame({
    'x1': np.random.uniform(0, 5, 20),
    'x2': np.random.uniform(0, 5, 20)
})

# Add noise to validation data to make plots interesting
# Amplitude is roughly 3 to 30, so noise of 1.0 is visible but not overwhelming
Y_val_1 = generate_data_from_func(X_val, y1_func, noise_std=1.5)
Y_val_2 = generate_data_from_func(X_val, y2_func, noise_std=0.5) # Smaller noise for smaller amplitude signal

ds_val_1 = DatasetConfig(
    name='Dataset1',
    data=Y_val_1
)

ds_val_2 = DatasetConfig(
    name='Dataset2',
    data=Y_val_2
)

# Run Validation
val_result = q.validate(X_val, [ds_val_1, ds_val_2])

# Show Metrics Summary
print(val_result.summary())

              RMSE       MAE  Peak Error     SRMSE
Dataset                                           
Dataset1  1.605251  1.305253    1.468734  0.286478
Dataset2  0.517200  0.417442    0.573368  0.147594


## 4. Interactive Dashboard

Use the dashboard below to inspect the results. 
- **Parity Plot**: Click on any point to see the corresponding curve comparison.
- **Curve Comparison**: Shows the Actual (Black) vs Predicted (Blue Dashed) curves.

In [43]:
val_result.dashboard()

## 5. Export Report

Save the validation results to an HTML file for sharing.

In [ ]:
val_result.save_html('validation_report.html')
print('Report saved to validation_report.html')